In [1]:
import os, sys

fileDir = os.getcwd()
path = os.path.join(fileDir, "..")
sys.path.append(path)

from utilities import *

# Sample Data

In [2]:
data = [("1", "Alice"),
        ("2", "Bob"),
        ("3", "Charlie")]

relationship = [("Person/1", "Person/2"), 
                ("Person/2", "Person/3")]

columns = ["_key", "name"]
relationship_columns = ["_from", "_to"]

# ArangoDB connector test

In [3]:
session = create_spark_session("ArangoDB Test", SparkConnector.ARANGO)

Added dependencies: 
 ['arangodb-java-driver-shaded-7.1.0.jar', 'arangodb-spark-commons-3.3_2.12-1.5.0.jar', 'arangodb-spark-datasource-3.3_2.12-1.5.0.jar', 'commons-codec-1.11.jar', 'commons-logging-1.2.jar', 'httpclient-4.5.13.jar', 'httpcore-4.4.13.jar', 'jackson-dataformat-velocypack-4.1.0.jar', 'slf4j-api-2.0.7.jar']


## Write

In [4]:
df = session.createDataFrame(data, columns)

options = get_default_options(SparkConnector.ARANGO)
options["table.type"] = "document"
options["table"] = "Person"
options["createCollection"] = "true"

spark_write(SparkConnector.ARANGO, df, "Append", options=options)

Dataframe saved to ARANGO


In [5]:
df = session.createDataFrame(relationship, relationship_columns)
df = set_df_columns_nullable(session, df, ["_from", "_to"], False)

options = get_default_options(SparkConnector.ARANGO)
options["table.type"] = "edge"
options["table"] = "FRIEND"
options["createCollection"] = "true"

spark_write(SparkConnector.ARANGO, df, "Append", options=options)

Dataframe saved to ARANGO


## Read

In [6]:
options = get_default_options(SparkConnector.ARANGO)
options["table"] = "Person"

df = spark_read(SparkConnector.ARANGO, session, options=options)
df.show()

Dataframe loaded from arangodb
+--------+----+-----------+-------+
|     _id|_key|       _rev|   name|
+--------+----+-----------+-------+
|Person/1|   1|_ge2at3G---|  Alice|
|Person/2|   2|_ge2at5----|    Bob|
|Person/3|   3|_ge2at5---_|Charlie|
+--------+----+-----------+-------+



In [7]:
options = get_default_options(SparkConnector.ARANGO)
options["query"] = "for v in Person \
                    for f in outbound v FRIEND \
                    return {'name': v.name, 'friend': f.name}"

df = spark_read(SparkConnector.ARANGO, session, options=options)
df.show()

Dataframe loaded from arangodb
+-------+-----+
| friend| name|
+-------+-----+
|    Bob|Alice|
|Charlie|  Bob|
+-------+-----+



## Stop spark context and spark session

In [8]:

session.sparkContext.stop()
session.stop()